In [ ]:
import os 
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr
import json

In [ ]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key and openai_api_key.startswith('sk-proj-'):
    print('API Key exists, good to go')
else:
    print('API Key doesnt exist, troubleshoot the problem!')

In [ ]:
openai = OpenAI()
MODEL = 'gpt-4o-mini'

system_prompt =  "You are a helpful assistant for an Airline called FlighAI. "
system_prompt += "Give short, courteous answers, not more than 1 sentence. "
system_prompt += "Always be accurate. If you don't know the answer, say so."


In [ ]:
def get_ticket_prices(destination_city):
    ticket_prices = {'tokyo': '1499', 'paris': '599', 'sydney': '499', 'berlin': '799'}
    city = destination_city.lower()
    return ticket_prices.get(city, 'Unknown')

In [ ]:
price_tool_function = {
    "name": "get_ticket_prices",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to."
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
tool = [{'type': 'function', 'function': price_tool_function}]

In [ ]:
def handle_tool_call(assistant_message):
    tool_call = assistant_message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_prices(city)
    tool_response = {
        'role': 'tool',
        'content': json.dumps({"destination_city": city, "price": price}),
        'tool_call_id': tool_call.id
    }
    return tool_response, city

In [ ]:
def airline_customer_support_agent(user_prompt, history):
    messages = [{'role': 'system', 'content': system_prompt}] + history + [{'role': 'user', 'content': user_prompt}]
    response = openai.chat.completions.create(
        model= MODEL,
        messages= messages,
        temperature= 0.5,
        tools= tool
    )

    if response.choices[0].finish_reason == 'tool_calls':
        assistant_response = response.choices[0].message
        tool_response,city = handle_tool_call(assistant_response)
        messages.append(assistant_response)
        messages.append(tool_response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(fn= airline_customer_support_agent, type= 'messages').launch()